In [1]:
import torch
import numpy as np
# from utils import compute_ideal_dcg, ndcg
from lecture2_task import compute_ideal_dcg, ndcg

In [2]:
# в y_true лежат оценки релевантности
y_true = torch.LongTensor([[5, 3, 2, 5, 1, 1]]).reshape(-1, 1)
y_pred = torch.FloatTensor([3.2, 0.4, -0.1, -2.1, 0.5, 0.01]).reshape(-1, 1)

In [3]:
y_pred

tensor([[ 3.2000],
        [ 0.4000],
        [-0.1000],
        [-2.1000],
        [ 0.5000],
        [ 0.0100]])

In [4]:
%%latex
$$\lambda = \left(0.5 * (1 - S_{ij}) - \frac {1} {1 + e^{s_i - s_j}}\right) |\Delta nDCG|$$

<IPython.core.display.Latex object>

In [5]:
%%latex
$$\Delta nDCG = \frac {1} {IdealDCG} (2^i - 2^j) \left(\frac {1} {log_2(1+i)} - \frac {1} {log_2(1+j)}\right)$$

<IPython.core.display.Latex object>

In [6]:
def compute_lambdas(y_true, y_pred, ndcg_scheme='exp2'):
    # рассчитаем нормировку, IdealDCG
    ideal_dcg = compute_ideal_dcg(y_true, ndcg_scheme=ndcg_scheme)
    N = 1 / ideal_dcg
    
    # рассчитаем порядок документов согласно оценкам релевантности
    _, rank_order = torch.sort(y_true, descending=True, axis=0)
    rank_order += 1
    
    with torch.no_grad():
        # получаем все попарные разницы скоров в батче
        pos_pairs_score_diff = 1.0 + torch.exp((y_pred - y_pred.t())) # транспорируем. получаем broadcast - попрная разность 
                                                                         # (torch додумал что мы хотчел сделать)
        # 1 со 2,3,4,... 2 с 3,..
        
        # поставим разметку для пар, 1 если первый документ релевантнее
        # -1 если второй документ релевантнее
        Sij = compute_labels_in_batch(y_true)
        # посчитаем изменение gain из-за перестановок
        gain_diff = compute_gain_diff(y_true, ndcg_scheme)
        
        # посчитаем изменение знаменателей-дискаунтеров (последний множитель)
        decay_diff = (1.0 / torch.log2(rank_order + 1.0)) - (1.0 / torch.log2(rank_order.t() + 1.0))
        # посчитаем непосредственное изменение nDCG
        delta_ndcg = torch.abs(N * gain_diff * decay_diff)
        # посчитаем лямбды
        lambda_update =  (0.5 * (1 - Sij) - 1 / pos_pairs_score_diff) * delta_ndcg
        lambda_update = torch.sum(lambda_update, dim=1, keepdim=True) # суммируем вдоль строчек
        # целевое изменение скора чтобы наша метрика улучшалась
        
        return Sij, gain_diff, decay_diff, delta_ndcg, lambda_update
    

    
def compute_labels_in_batch(y_true):
    
    # разница релевантностей каждого с каждым объектом
    rel_diff = y_true - y_true.t()
    
    # 1 в этой матрице - объект более релевантен
    pos_pairs = (rel_diff > 0).type(torch.float32)
    
    # 1 тут - объект менее релевантен
    neg_pairs = (rel_diff < 0).type(torch.float32)
    Sij = pos_pairs - neg_pairs
    return Sij

def compute_gain_diff(y_true, gain_scheme):
    if gain_scheme == "exp2":
        gain_diff = torch.pow(2.0, y_true) - torch.pow(2.0, y_true.t())
    elif gain_scheme == "diff":
        gain_diff = y_true - y_true.t()
    else:
        raise ValueError(f"{gain_scheme} method not supported")
    return gain_diff

In [7]:
y_pred - y_pred.t()
# первый документ релевантнее второго на 2.8

tensor([[ 0.0000,  2.8000,  3.3000,  5.3000,  2.7000,  3.1900],
        [-2.8000,  0.0000,  0.5000,  2.5000, -0.1000,  0.3900],
        [-3.3000, -0.5000,  0.0000,  2.0000, -0.6000, -0.1100],
        [-5.3000, -2.5000, -2.0000,  0.0000, -2.6000, -2.1100],
        [-2.7000,  0.1000,  0.6000,  2.6000,  0.0000,  0.4900],
        [-3.1900, -0.3900,  0.1100,  2.1100, -0.4900,  0.0000]])

In [8]:
y_true - y_true.t()

tensor([[ 0,  2,  3,  0,  4,  4],
        [-2,  0,  1, -2,  2,  2],
        [-3, -1,  0, -3,  1,  1],
        [ 0,  2,  3,  0,  4,  4],
        [-4, -2, -1, -4,  0,  0],
        [-4, -2, -1, -4,  0,  0]])

In [9]:
Sij, gain_diff, decay_diff, delta_ndcg, lambda_update = compute_lambdas(y_true, y_pred)

In [10]:
Sij
# 5й не релевантнее себя и 4го док-та (у него такая же релевантность) и релевантнее всех остальных документов - 1 строка

tensor([[ 0.,  1.,  1.,  0.,  1.,  1.],
        [-1.,  0.,  1., -1.,  1.,  1.],
        [-1., -1.,  0., -1.,  1.,  1.],
        [ 0.,  1.,  1.,  0.,  1.,  1.],
        [-1., -1., -1., -1.,  0.,  0.],
        [-1., -1., -1., -1.,  0.,  0.]])

In [11]:
gain_diff

tensor([[  0.,  24.,  28.,   0.,  30.,  30.],
        [-24.,   0.,   4., -24.,   6.,   6.],
        [-28.,  -4.,   0., -28.,   2.,   2.],
        [  0.,  24.,  28.,   0.,  30.,  30.],
        [-30.,  -6.,  -2., -30.,   0.,   0.],
        [-30.,  -6.,  -2., -30.,   0.,   0.]])

In [13]:
# пример вычисления элемента gain diff для первого (релевантность 5) и последнего документа (1); 
# для первого (5) и второго (3) документа
# delta gain из-за перестановок объектов в ранжировании
(2**5 - 1) - (2**1 -1), (2**5 - 1) - (2**3-1)

(30, 24)

In [14]:
decay_diff

tensor([[ 0.0000,  0.5693,  0.3691,  0.5000,  0.6131,  0.6438],
        [-0.5693,  0.0000, -0.2003, -0.0693,  0.0438,  0.0745],
        [-0.3691,  0.2003,  0.0000,  0.1309,  0.2441,  0.2747],
        [-0.5000,  0.0693, -0.1309,  0.0000,  0.1131,  0.1438],
        [-0.6131, -0.0438, -0.2441, -0.1131,  0.0000,  0.0306],
        [-0.6438, -0.0745, -0.2747, -0.1438, -0.0306,  0.0000]])

In [15]:
# посчитаем изменение знаменателей-дискаунтеров для первого и последнего документа
(1 / np.log2(1+1)) - (1 / np.log2(1+6))

0.6437928128919779

In [16]:
delta_ndcg

tensor([[0.0000, 0.2436, 0.1842, 0.0000, 0.3279, 0.3443],
        [0.2436, 0.0000, 0.0143, 0.0297, 0.0047, 0.0080],
        [0.1842, 0.0143, 0.0000, 0.0654, 0.0087, 0.0098],
        [0.0000, 0.0297, 0.0654, 0.0000, 0.0605, 0.0769],
        [0.3279, 0.0047, 0.0087, 0.0605, 0.0000, 0.0000],
        [0.3443, 0.0080, 0.0098, 0.0769, 0.0000, 0.0000]])

In [17]:
lambda_update
# изменение скоров для каждого документа

tensor([[-0.0548],
        [ 0.0303],
        [ 0.0587],
        [-0.2099],
        [ 0.0851],
        [ 0.0906]])

In [18]:
for _ in range(100):
    _, _, _, _, lambda_update = compute_lambdas(y_true, y_pred)
    y_pred -= lambda_update

In [19]:
rank_indexes = torch.argsort(y_pred, dim=0, descending=True)

In [20]:
y_pred

tensor([[ 4.3905],
        [-0.1219],
        [-1.0463],
        [ 2.0759],
        [-1.5135],
        [-1.8746]])

In [21]:
y_true[rank_indexes]

tensor([[[5]],

        [[5]],

        [[3]],

        [[2]],

        [[1]],

        [[1]]])

In [22]:
# полностью правильное ранжирование
torch.sort(y_true, dim=0, descending=True)[0]

tensor([[5],
        [5],
        [3],
        [2],
        [1],
        [1]])

In [23]:
y_true = torch.LongTensor([[5,3,2,5,1,1]]).reshape(-1,1)

# совсем плохие предсказанные скоры в начале
y_pred = torch.FloatTensor([-3.0, 2.0, 3.0, -4.0, 6.0, 8.5]).reshape(-1,1)

In [24]:
ndcg(y_true, y_pred)

0.6784099316652937

In [28]:
for _ in range(100):
    _, _, _, _, lambda_update = compute_lambdas(y_true, y_pred)
    y_pred -= lambda_update

In [29]:
ndcg(y_true, y_pred)

1.0

In [30]:
# полностью правильное ранжирование при увеличении количества итераций
y_pred

tensor([[6.3082],
        [1.2822],
        [0.5772],
        [3.7786],
        [0.2697],
        [0.2841]])